In [2]:
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

clf_multilabel = OneVsRestClassifier(LGBMClassifier(),n_jobs = 4)


In [3]:

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense
import numpy as np

df = pd.read_csv(r'C:\Users\pinkpigma\pinkpigma的同步盘\KDD研二上\可解释性框架-工作\数据\patient_diagnoses2000_ndc300_with_history_delete2.csv')
# 预测第二天用药，第一天当做历史信息
x = df.iloc[:,5:5+72+2000+300]
y = df.iloc[:,5+72+2000+300:5+72+2000+300+300]


# 将字符串类型的数据转化为数值
from sklearn. preprocessing import LabelEncoder 
x['gender'] = LabelEncoder().fit_transform(x['gender'])
x['ethnicity'] = LabelEncoder().fit_transform(x['ethnicity'])
x['ethnicity_grouped'] = LabelEncoder().fit_transform(x['ethnicity_grouped'])
x['first_hosp_stay'] = LabelEncoder().fit_transform(x['first_hosp_stay'])


# 填补缺失值，用平均值
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
x = imp.fit_transform(x)
y = np.array(y)

# 划分数据集 9:1
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=2)
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [4]:


clf_multilabel.fit(X_train,y_train)


OneVsRestClassifier(estimator=LGBMClassifier(), n_jobs=4)

In [5]:

val_pred = clf_multilabel.predict(X_test)

In [9]:
train_pred= clf_multilabel.predict(X_train)

In [6]:
def jaccard(y_true, y_pred):
    fenzi=0
    fenmu=0
    for i in range(len(y_true)):
        fenzi += y_true[i] & y_pred[i]
        fenmu += y_true[i] | y_pred[i]
    return fenzi/fenmu
    

In [7]:
JI = 0
for i in range(len(y_test)):
    JI += jaccard(y_test[i], val_pred[i])
JI/len(y_test)




0.10252420685919524

In [10]:
JI = 0
for i in range(len(y_train)):
    JI += jaccard(y_train[i], train_pred[i])
JI/len(y_train)



0.6101712771197129

In [11]:
def jaccard(y_gt, y_pred):
    score = []
    for b in range(y_gt.shape[0]):
        target = np.where(y_gt[b] == 1)[0]
        out_list = np.where(y_pred[b] == 1)[0]
        inter = set(out_list) & set(target)
        union = set(out_list) | set(target)
        jaccard_score = 0 if union == 0 else len(inter) / len(union)
        score.append(jaccard_score)
    return np.mean(score)

In [12]:
jaccard(y_test, val_pred)

0.1025242068591953

In [13]:
def average_prc(y_gt, y_pred):
    score = []
    for b in range(y_gt.shape[0]):
        target = np.where(y_gt[b] == 1)[0]
        out_list = np.where(y_pred[b] == 1)[0]
        inter = set(out_list) & set(target)
        prc_score = 0 if len(out_list) == 0 else len(inter) / len(out_list)
        score.append(prc_score)
    return score